In [1]:
import zipfile
import json
import polars as pl
import os

# CONFIGURACIÓN
ZIP_PATH = CARPETA_DATA = r"./Data/" + "esports_data.zip" # <--- Tu nuevo archivo ZIP
OUTPUT_META_FILE = "draft_oracle_tournament_meta.parquet"

def process_tournament_data():
    print("🏆 Analizando Meta de Torneos (Estructura Complex JSON)...")

    if not os.path.exists(ZIP_PATH):
        print(f"⚠️ No encuentro {ZIP_PATH}. Súbelo primero.")
        return

    data_rows = []

    # Índices de Draft que corresponden a BANS en formato Torneo Estándar
    # Fase 1 Bans: 1-6 | Fase 2 Bans: 13-16
    BAN_INDICES = {1, 2, 3, 4, 5, 6, 13, 14, 15, 16}

    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        # Buscar solo los archivos series_details.json
        json_files = [f for f in z.namelist() if f.endswith("series_details.json")]
        print(f"   📄 Procesando {len(json_files)} series encontradas...")

        for json_file in json_files:
            try:
                with z.open(json_file) as f:
                    series_data = json.load(f)

                    # El root suele ser una lista de series dicts [ {...}, {...} ]
                    # O a veces un dict directo. Nos aseguramos iterando si es lista.
                    if isinstance(series_data, dict): series_list = [series_data]
                    else: series_list = series_data

                    for series in series_list:
                        games = series.get("games", [])

                        for game in games:
                            teams = game.get("teams", [])
                            if len(teams) < 2: continue

                            # 1. IDENTIFICAR GANADOR (Quien destruyó el Nexo)
                            winner_name = None
                            for team in teams:
                                objectives = team.get("objectives", [])
                                # Buscamos si este equipo rompió el nexo
                                if any(obj.get("type") == "destroyNexus" for obj in objectives):
                                    winner_name = team.get("name")
                                    break

                            # Si no encontramos destroyNexus, probamos lógica fallback (won en series header)
                            # Pero asumiremos destroyNexus es fiable para game-level data.

                            # 2. PROCESAR PICKS (Desde 'players')
                            for team in teams:
                                team_name = team.get("name")
                                is_winner = (team_name == winner_name)

                                for player in team.get("players", []):
                                    char_info = player.get("character", {})
                                    c_name = char_info.get("name")

                                    if c_name:
                                        data_rows.append({
                                            "champion": c_name,
                                            "type": "PICK",
                                            "win": 1 if is_winner else 0
                                        })

                            # 3. PROCESAR BANS (Desde 'draftActions')
                            actions = game.get("draftActions", [])
                            for action in actions:
                                # sequenceNumber viene como string "1", hay que convertir
                                try:
                                    seq = int(action.get("sequenceNumber", 0))
                                except: continue

                                if seq in BAN_INDICES:
                                    # Es un BAN
                                    draftable = action.get("draftable", {})
                                    c_name = draftable.get("name")
                                    if c_name:
                                        data_rows.append({
                                            "champion": c_name,
                                            "type": "BAN",
                                            "win": 0 # Da igual en bans
                                        })

            except Exception as e:
                # print(f"Error leyendo {json_file}: {e}") # Descomentar para debug
                continue

    # --- GENERAR PARQUET AGREGADO ---
    if not data_rows:
        print("❌ No se extrajeron datos. Revisa si el ZIP es correcto.")
        return

    print(f"   📊 Datos brutos extraídos: {len(data_rows)} picks/bans.")

    df = pl.DataFrame(data_rows)

    # Agregación por Campeón
    meta_stats = df.group_by("champion").agg([
        pl.col("type").filter(pl.col("type") == "PICK").count().alias("tourney_picks"),
        pl.col("type").filter(pl.col("type") == "BAN").count().alias("tourney_bans"),
        # Winrate calculado solo sobre los PICKS
        pl.col("win").filter(pl.col("type") == "PICK").mean().fill_null(0).alias("tourney_winrate")
    ])

    # Calcular Presencia Total
    meta_stats = meta_stats.with_columns(
        (pl.col("tourney_picks") + pl.col("tourney_bans")).alias("tourney_presence"),
        pl.col("champion").str.to_lowercase().alias("champ_key")
    )

    # Guardar
    print(f"   💾 Guardando Meta: {meta_stats.height} campeones.")
    meta_stats.write_parquet(OUTPUT_META_FILE)

    # Mostrar Top 5 del Meta
    print("\n👑 TOP META (PRESENCIA):")
    print(meta_stats.sort("tourney_presence", descending=True).head(5))

# EJECUTAR
process_tournament_data()

🏆 Analizando Meta de Torneos (Estructura Complex JSON)...
   📄 Procesando 78 series encontradas...
   📊 Datos brutos extraídos: 67808 picks/bans.
   💾 Guardando Meta: 162 campeones.

👑 TOP META (PRESENCIA):
shape: (5, 6)
┌──────────┬───────────────┬──────────────┬─────────────────┬──────────────────┬───────────┐
│ champion ┆ tourney_picks ┆ tourney_bans ┆ tourney_winrate ┆ tourney_presence ┆ champ_key │
│ ---      ┆ ---           ┆ ---          ┆ ---             ┆ ---              ┆ ---       │
│ str      ┆ u32           ┆ u32          ┆ f64             ┆ u32              ┆ str       │
╞══════════╪═══════════════╪══════════════╪═════════════════╪══════════════════╪═══════════╡
│ Rumble   ┆ 736           ┆ 1385         ┆ 0.523098        ┆ 2121             ┆ rumble    │
│ Vi       ┆ 639           ┆ 1383         ┆ 0.499218        ┆ 2022             ┆ vi        │
│ Varus    ┆ 762           ┆ 1206         ┆ 0.513123        ┆ 1968             ┆ varus     │
│ Kalista  ┆ 382           ┆ 1425  